In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
data = pd.read_csv('./test5-beam/source.dat', delim_whitespace=True, header=None)
data.columns = ['particle_type', 'momentum', 'energy', 'dx', 'dy', 'dz', 'x', 'y', 'z']

# fluka上的地球半径 
R = 6378.14  
# 转到xyz坐标的一个函数
def latlon_to_xyz(lat, lon, radius=R):
    lat = np.radians(lat)
    lon = np.radians(lon)
    x = radius * np.cos(lat) * np.cos(lon)
    y = radius * np.cos(lat) * np.sin(lon)
    z = radius * np.sin(lat)
    return x, y, z
lat_ref, lon_ref = 11.07, -177.25 #22.86, 113.92
x_ref, y_ref, z_ref = latlon_to_xyz(lat_ref, lon_ref)
# 粒子的颜色映射
particle_colors = {
    -6: ('red', 'Helium-4'),        # helium-4
    -2: ('orange', 'Heavy Ion'),    # HEAVYION
    1: ('blue', 'Proton'),          # PROTON
    2: ('cyan', 'Anti-Proton'),     # APROTON
    8: ('purple', 'Neutron'),       # NEUTRON
    10: ('green', 'Muon+'),         # MUON+
    11: ('darkgreen', 'Muon-'),     # MUON-
    13: ('pink', 'Pion+'),          # PION+
}

    # 3:  'gray',        # dian
    # 7:  'gold'        #aotoman


fig = go.Figure()
# 先根据fluka半径画一个地球#抄csdn
theta = np.linspace(0, np.pi, 100)
phi = np.linspace(0, 2 * np.pi, 100)
theta, phi = np.meshgrid(theta, phi)
x_sphere = R * np.sin(theta) * np.cos(phi)
y_sphere = R * np.sin(theta) * np.sin(phi)
z_sphere = R * np.cos(theta)
fig.add_trace(go.Mesh3d(
    x=x_sphere.ravel(),
    y=y_sphere.ravel(),
    z=z_sphere.ravel(),
    opacity=0.15,           
    color='pink',           
    name="Earth"
))


# 换KM,因为fluka数据中单位都是cm
scale_position = 0.00001
# arrow_length = 0.2
for particle_type, (color, name) in particle_colors.items():
    subset = data[data['particle_type'] == particle_type]
    if not subset.empty:
        fig.add_trace(go.Scatter3d(
            x=subset['x'] * scale_position,
            y=subset['y'] * scale_position,
            z=subset['z'] * scale_position,
            mode='markers',
            marker=dict(size=2, color=color, opacity=0.8),
            name=name
        ))
        
        # norms = np.sqrt(subset['dx']**2 + subset['dy']**2 + subset['dz']**2)
        # u = (subset['dx'] / norms) * arrow_length
        # v = (subset['dy'] / norms) * arrow_length
        # w = (subset['dz'] / norms) * arrow_length
        # fig.add_trace(go.Cone(
        #     x=subset['x'] * scale_position,
        #     y=subset['y'] * scale_position,
        #     z=subset['z'] * scale_position,
        #     u=u,
        #     v=v,
        #     w=w,
        #     colorscale=[[0, color], [1, color]],
        #     showscale=False,
        #     sizemode='absolute',
        #     sizeref=0.2
        # ))
# 东莞位置
fig.add_trace(go.Scatter3d(
    x=[x_ref], y=[y_ref], z=[z_ref],
    mode='markers',
    marker=dict(size=5, color='black', opacity=0.87),
    name=f'my position ({lat_ref}°, {lon_ref}°)'
))
fig.update_layout(
    scene=dict(
        xaxis=dict(title='X', backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(title='Y', backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(title='Z', backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='cube',  
        xaxis_range=[-R * 1.2, R * 1.2],
        yaxis_range=[-R * 1.2, R * 1.2],
        zaxis_range=[-R * 1.2, R * 1.2]
    ),
    title="3D Particle Locations with Earth Sphere",
    showlegend=True,
    legend=dict(
        font=dict(size=12),    
        itemsizing='constant'  # 图例项的大小一致
    )
)

fig.show(renderer="browser")
# for column in data.columns:
#     print(f'{column}:')
#     print(data[column].head(20))
#     print()  

# for particle_type in particle_colors.keys():
#     subset = data[data['particle_type'] == particle_type]
#     print(f"Particle Type {particle_type} dx, dy, dz:\n", subset[['dx', 'dy', 'dz']].head())

print(x_ref, y_ref, z_ref)